In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = "/content/drive/MyDrive/연구실/"

In [3]:
!pip install git+https://github.com/hyperopt/hyperopt.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/hyperopt/hyperopt.git to /tmp/pip-req-build-cp5cs4yl
  Running command git clone --filter=blob:none --quiet https://github.com/hyperopt/hyperopt.git /tmp/pip-req-build-cp5cs4yl
  Resolved https://github.com/hyperopt/hyperopt.git to commit 6c5b33be93a71a4ae6a38f04a29b18020b78b702
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 7.6 MB/s eta 0:00:00
  Created wheel for hyperopt: filename=hyperopt-0.2.7-py2.py3-none-any.whl size=966138 sha256=a7f3ca5b81af052c8a26dd0fce6001022a3da18ba4beb511e2e58d8a33b28d20
  Stored in directory: /tmp/pip-ephem-wheel-cache-u8zucvld/wheels/7e/a2/bc/da25ffbedcf76e6957370dd98f8bfc0b9cda94cef6c350d117
Successfully built hyperopt
  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.1.2
    Uninstallin

In [4]:
import pandas as pd
import numpy as np

from hyperopt import hp
from hyperopt import STATUS_OK
from hyperopt.early_stop import no_progress_loss

from sklearn.cross_decomposition import PLSRegression
from tqdm import tqdm
import joblib

import warnings
warnings.filterwarnings("ignore")

In [5]:
data = pd.read_csv(path+'전처리최종이었음좋겠다.csv')
data.columns

Index(['yearmonth', 'Ticker', '한국표준산업분류코드10차(중분류)', '수익률 (1개월)(%)',
       '베타 (D,1Yr)', 'PER(배)', 'PSR(배)', 'chmom', 'mom36m', 'zerotrade',
       'std_turn', 'mean_ill', 'retvol', 'maxret', 'var_market', 'mom1m',
       'mom12m', 'indmom', 'turn', 'mvel1', 'dolvol', 'betasq', 'Idiovol',
       'macro_(콜금리-기준금리)평균', 'macro_원/달러환율', 'macro_금($/OZ)', 'macro_뉴스심리지수',
       'macro_KOSPI dp', 'macro_KOSPI PER', 'macro_KOSPI PBR', 'macro_CD Rate',
       'macro_Term spread', 'macro_Default Spread', 'macro_VKOSPI 200'],
      dtype='object')

In [6]:
X = data.drop(['한국표준산업분류코드10차(중분류)', '수익률 (1개월)(%)', 'macro_뉴스심리지수'], axis=1)
y = data['수익률 (1개월)(%)']
ym = data['yearmonth']
tk = data['Ticker']

In [7]:
X.shape

(312185, 31)

In [8]:
a = '2004|2005|2006|2007|2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020'
ya0 = []
tk0 = []
ym0 = []
ya1 = []
sse = 0

for i in tqdm(range(9)): 
    # tX = X[ym.str.contains(a[:24+5*i])]
    # vX = X[ym.str.contains(a[25+5*i:39+5*i])]
    tvX = X[ym.str.contains(a[:39+5*i])]
    tvym = ym[ym.str.contains(a[:39+5*i])]
    oX = X[ym.str.contains(a[40+5*i:44+5*i])]
    # ty = y[ym.str.contains(a[:24+5*i])]
    Vy = y[ym.str.contains(a[25+5*i:39+5*i])]
    tvy = y[ym.str.contains(a[:39+5*i])]
    oy = y[ym.str.contains(a[40+5*i:44+5*i])]
    otk = tk[ym.str.contains(a[40+5*i:44+5*i])]
    oym = ym[ym.str.contains(a[40+5*i:44+5*i])]
    
    # 하이퍼파라미터 검색 공간 설정
    pls_search_space = {
        'n_components' : hp.quniform('n_components', 1, 29, 1)
    }
    
    # 목적 함수 정의
    def objective_func(search_space):
        pls = PLSRegression(
            n_components = int(search_space['n_components'])
        )
        
        d = 0
        
        # time series cv
        for j in range(3): # 마지막 3년만 valid로 사용
            tX = X[ym.str.contains(a[:24+5*(j+i)])]
            vX = X[ym.str.contains(a[25+5*(j+i):29+5*(j+i)])]
            ty = y[ym.str.contains(a[:24+5*(j+i)])]
            vy = y[ym.str.contains(a[25+5*(j+i):29+5*(j+i)])]
            
            # for문에 필요한 key를 미리 만듦
            ym_temp = vX['yearmonth'].reset_index(drop=True) # t = ym_temp==k를 위해 reset index
            key_yearmonth = dict.fromkeys(ym_temp)
            
            # vy의 index를 reset
            vy.reset_index(drop=True, inplace=True) # r = r*(vy[sort[-d:].index].mean()+100)/100를 위해 reset index
            
            # 년도, 티커 드랍
            tX = tX.drop(['yearmonth', 'Ticker'], axis=1)
            vX = vX.drop(['yearmonth', 'Ticker'], axis=1)
            
            # fitting
            pls.fit(tX, ty)
            
            # r2oos 계산         
            y_pred = pd.Series(pls.predict(vX).reshape(-1,)) # sort_values() 사용을 위해 변환
            d = d + sum((vy-y_pred)**2) 
                
        e = sum(Vy**2)
        r2oos = 1-(d/e)
        new_loss = r2oos*(-1)
        
        return {
            'loss':new_loss,
            'status':STATUS_OK
        }
    
    # fmin()을 사용하여 최적 하이퍼 파라미터 찾기
    from hyperopt import fmin, tpe, Trials
    import warnings
    warnings.filterwarnings('ignore')

    trial_val = Trials() # 결과 저장
    
    best = fmin(
        fn=objective_func,
        space=pls_search_space,
        algo=tpe.suggest,
        max_evals=1000,    # 최대 반복 횟수 지정
        trials=trial_val,
        return_argmin=False, # hp.choice의 경우 값 대신 index를 반환함 
        early_stop_fn=no_progress_loss(20), # 30회 동안 개선되지 않으면 조기중지
        # https://github.com/hyperopt/hyperopt/issues/689
        rstate=np.random.default_rng(42) # https://github.com/hyperopt/hyperopt/issues/838
    )
    
    # 정수형 하이퍼 파라미터 값이 뒤에 .0이 붙어서 실수형으로 반환됨을 유의하기!
    best['n_components'] = int(best['n_components'])
    print(best) 
    
    reg0 = PLSRegression(
        **best)
    
    import warnings
    warnings.filterwarnings('ignore')
    
    tvX = tvX.drop(['yearmonth', 'Ticker'], axis=1)
    oX = oX.drop(['yearmonth', 'Ticker'], axis=1)
    
    reg0.fit(tvX, tvy)
    
    yy0 = reg0.predict(oX).reshape(-1,)
    ya0 = ya0 + oy.tolist()
    ya1 = ya1 + yy0.tolist()
    tk0 = tk0 + otk.tolist()
    ym0 = ym0 + oym.tolist()
    yy = yy0.reshape(-1,)
    joblib.dump(reg0, path+'PLS_r2oos_'+str(i)+'.pkl')
    print(1-sum((oy-yy)**2)/sum(oy**2))
    sse = sse + sum((oy-yy)**2)

oyy = y[ym.str.contains(a[40:84])]
e = sum(oyy**2)
r2oos = 1-(sse/e)
print(r2oos)

PLS = pd.DataFrame({'real':ya0,'predict':ya1,'ticker':tk0,'yearmonth':ym0})
PLS.to_csv(path+'PLS_r2oos.csv')  

  0%|          | 0/9 [00:00<?, ?it/s]


  4%|▍         | 40/1000 [02:51<1:08:46,  4.30s/trial, best loss: 0.22749024939332774]
{'n_components': 14}


 11%|█         | 1/9 [02:57<23:37, 177.17s/it]

-0.004471534732645832

  4%|▍         | 41/1000 [02:53<1:07:32,  4.23s/trial, best loss: -0.01709377674378909]
{'n_components': 1}


 22%|██▏       | 2/9 [05:53<20:36, 176.62s/it]

...

 89%|████████▉ | 8/9 [29:26<04:08, 248.49s/it]

-0.04645945737198698

  4%|▍         | 45/1000 [05:24<1:54:41,  7.21s/trial, best loss: 0.022769803373485775]
{'n_components': 1}


100%|██████████| 9/9 [34:53<00:00, 232.57s/it]

0.02716656515185467


-0.0008993454872434636


In [9]:
ya0 = pd.Series(PLS['real'])
ya1 = pd.Series(PLS['predict'])
tk0 = pd.Series(PLS['ticker'])
ym0 = pd.Series(PLS['yearmonth'])

r1 = []
r2 = []
for i in list(dict.fromkeys(ym0)):
    t = ym0==i
    d = round(len(ya1[t])/10)
    sort = ya1[t].sort_values()
    r1.append(ya0[sort[-d:].index].mean())
    r2.append(ya0[sort[0:d].index].mean())

rr1 = 1
rpls1 = [1]
for i in r1:
    rr1 = rr1*(i+100)/100
    rpls1.append(rr1)

In [10]:
rpls1[-1]

15.010622170977351